In [61]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis2' from '/home/jovyan/data-analyses/rt_delay/rt_analysis2.py'>

In [62]:
analysis_date = dt.date(2022, 1, 11)

In [63]:
samtrans_itp_id = 290

In [64]:
sam_vp = utils.get_vehicle_postions(samtrans_itp_id, analysis_date)

In [65]:
sam_trips = utils.get_trips(samtrans_itp_id, analysis_date)

In [66]:
sam_trips >> head(5)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at
0,290,1,2022-01-11,5696249739208797591,11032514-136-Blocks-Weekday-54,294,0,2940192,2022-01-05,2099-01-01
1,290,1,2022-01-11,-295766390093355594,11031852-136-Blocks-Weekday-54,296,1,SamTrans79:2960306,2022-01-05,2099-01-01
2,290,1,2022-01-11,-1895078399495595091,11031695-136-Blocks-Weekday-54,ECR,1,SamTrans79:ECR1072,2022-01-05,2099-01-01
3,290,1,2022-01-11,807523176375654463,11031875-136-Blocks-Weekday-54,296,1,SamTrans79:2960306,2022-01-05,2099-01-01
4,290,1,2022-01-11,-3463984472723802600,11031210-136-Blocks-Weekday-54,292,1,2921167,2022-01-05,2099-01-01


In [67]:
sam_vp >> head(5)

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,290,1,1,2900,11344573-136-Blocks-Weekday-54-SDon-1111000,2022-01-10 15:59:38,37.474150,-122.437060,2022-01-10 16:00:19
1,290,1,10,718,11031370-136-Blocks-Weekday-54,2022-01-10 15:59:38,37.445880,-122.264550,2022-01-10 16:00:19
2,290,1,100,830,11031637-136-Blocks-Weekday-54,2022-01-10 15:59:38,37.491257,-122.240944,2022-01-10 16:00:19
3,290,1,101,833,11402708-137-Blocks-Weekday-14,2022-01-10 15:59:38,37.474040,-122.218185,2022-01-10 16:00:19
4,290,1,102,817,11402707-137-Blocks-Weekday-14,2022-01-10 15:59:38,37.530254,-122.288605,2022-01-10 16:00:19


In [68]:
sam_vp >> inner_join(_, sam_trips, on ='trip_id')

,calitp_itp_id_x,calitp_url_number_x,entity_id,vehicle_id,trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,calitp_itp_id_y,calitp_url_number_y,service_date,trip_key,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at
0,290,1,1,2900,11344573-136-Blocks-Weekday-54-SDon-1111000,2022-01-10 15:59:38,37.474150,-122.437060,2022-01-10 16:00:19,290,1,2022-01-11,-6179750531392898744,18,1,SamTrans79:180039,2022-01-05,2099-01-01
1,290,1,1,2900,11344573-136-Blocks-Weekday-54-SDon-1111000,2022-01-10 16:00:10,37.474150,-122.437060,2022-01-10 16:00:39,290,1,2022-01-11,-6179750531392898744,18,1,SamTrans79:180039,2022-01-05,2099-01-01
2,290,1,1,2900,11344573-136-Blocks-Weekday-54-SDon-1111000,2022-01-10 16:00:43,37.477837,-122.440890,2022-01-10 16:01:03,290,1,2022-01-11,-6179750531392898744,18,1,SamTrans79:180039,2022-01-05,2099-01-01
3,290,1,1,2900,11344573-136-Blocks-Weekday-54-SDon-1111000,2022-01-10 16:01:35,37.477837,-122.440890,2022-01-10 16:01:59,290,1,2022-01-11,-6179750531392898744,18,1,SamTrans79:180039,2022-01-05,2099-01-01
4,290,1,1,2900,11344573-136-Blocks-Weekday-54-SDon-1111000,2022-01-10 16:01:58,37.480260,-122.443310,2022-01-10 16:02:22,290,1,2022-01-11,-6179750531392898744,18,1,SamTrans79:180039,2022-01-05,2099-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126022,290,1,65,830,11031687-136-Blocks-Weekday-54,2022-01-12 15:37:54,37.443188,-122.165060,2022-01-12 15:38:10,290,1,2022-01-11,-1622293963409416755,ECR,0,SamTrans79:ECR1063,2022-01-05,2099-01-01
126023,290,1,65,830,11031687-136-Blocks-Weekday-54,2022-01-12 15:38:27,37.443794,-122.166046,2022-01-12 15:38:55,290,1,2022-01-11,-1622293963409416755,ECR,0,SamTrans79:ECR1063,2022-01-05,2099-01-01
126024,290,1,66,830,11031687-136-Blocks-Weekday-54,2022-01-12 15:39:07,37.443794,-122.166046,2022-01-12 15:39:36,290,1,2022-01-11,-1622293963409416755,ECR,0,SamTrans79:ECR1063,2022-01-05,2099-01-01
126025,290,1,66,830,11031687-136-Blocks-Weekday-54,2022-01-12 15:39:43,37.443794,-122.166046,2022-01-12 15:40:10,290,1,2022-01-11,-1622293963409416755,ECR,0,SamTrans79:ECR1063,2022-01-05,2099-01-01


In [22]:
stop_times = (tbl.gtfs_schedule.stop_times()
     >> filter(_.calitp_itp_id == 170)
     # >> filter(_.trip_id == '9288623')
     >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
     >> arrange(_.stop_sequence)
     >> collect()
    )

In [23]:
min_date, max_date = (date_str, next_str)

sam_trips_query = (tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
    >> filter(_.calitp_itp_id == samtrans_itp_id)
    >> filter(_.service_date == date_str)
    >> filter(_.calitp_extracted_at == _.calitp_extracted_at.max())
    >> filter(_.is_in_service == True)
    >> select(_.trip_key, _.service_date)
    )

In [24]:
sam_trips_ix_query = (sam_trips_query
    >> inner_join(_, tbl.views.gtfs_schedule_index_feed_trip_stops(), on = 'trip_key')
    )

In [25]:
sam_st_query = (tbl.views.gtfs_schedule_dim_stop_times()
    >> filter(_.calitp_itp_id == samtrans_itp_id)
    )

In [26]:
sam_st = (sam_trips_ix_query
    >> inner_join(_, sam_st_query, on = 'stop_time_key')
    >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
    >> arrange(_.stop_sequence)
    >> collect()
    )

* it kind of seems that once you've got a key, filtering extracted/deleted is bad news...

In [31]:
stops = (tbl.views.gtfs_schedule_dim_stops()
     >> filter(_.calitp_itp_id == samtrans_itp_id)
     >> distinct(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
              _.stop_lat, _.stop_lon, _.stop_name, _.stop_key)
     # >> collect()
     # >> distinct(_.stop_key, _keep_all=True)
     >> inner_join(_, sam_trips_ix_query >> distinct(_.stop_key), on = 'stop_key')
     >> collect()
    )

stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                        crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [32]:
stops.head(3)

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,stop_key,geometry
0,290,1,311127,37.645975,-122.491054,Oceana Blvd & Milagra Dr,5220279661026859549,POINT (-219469.247 -38291.690)
1,290,1,311115,37.651201,-122.483763,Monterey Rd & Beaumont Blvd,7577499366621436568,POINT (-218811.817 -37727.788)
2,290,1,332003,37.684521,-122.479414,Alta Loma Ave & Eastmoor Ave,-1783174898912432600,POINT (-218331.914 -34035.026)


In [33]:
stops >> filter(_.stop_name.str.contains('Ruiss'))

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,stop_key,geometry
1206,290,1,315018,37.483849,-122.446537,Hwy 1 & Ruisseau Francais Ave,5626421419577972445,POINT (-216013.072 -56410.596)
1427,290,1,315017,37.483802,-122.446712,Hwy 1 & Ruisseau Francais Ave,7985544784696432888,POINT (-216028.655 -56415.420)
2543,290,1,315018,37.483849,-122.446537,Hwy 1 & Ruisseau Francais Ave,-4832955083364989897,POINT (-216013.072 -56410.596)
2802,290,1,315017,37.483802,-122.446712,Hwy 1 & Ruisseau Francais Ave,-1355362411902854211,POINT (-216028.655 -56415.420)
4890,290,1,315017,37.483802,-122.446712,Hwy 1 & Ruisseau Francais Ave,-8064029110528363578,POINT (-216028.655 -56415.420)
4954,290,1,315018,37.483849,-122.446537,Hwy 1 & Ruisseau Francais Ave,-6888104359049458766,POINT (-216013.072 -56410.596)


* we seem to have 3 rows per stop with different keys... (can't drop if we need keys to match with trips...)

In [131]:
trips_positions_joined = (samtrans_trips
                        >> inner_join(_, (samtrans_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id'])
                       )

In [133]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [134]:
trips_positions_joined.head(3)

,calitp_itp_id,calitp_url_number_x,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,calitp_url_number_y,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
0,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21,0,70,509,2021-11-29 15:59:33,37.534653,-122.300766,2021-11-29 16:00:10,POINT (-203008.021 -51086.885)
1,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21,0,68,509,2021-11-29 16:00:14,37.537376,-122.311630,2021-11-29 16:00:30,POINT (-203959.037 -50760.956)
2,290,1,2021-11-30,-8118108557453686259,11032442-136-Blocks-Weekday-54,250,1,2500398,2021-10-12,2021-12-21,0,68,509,2021-11-29 16:00:14,37.537376,-122.311630,2021-11-29 16:00:50,POINT (-203959.037 -50760.956)


In [137]:
st_routelines = shared_utils.geography_utils.make_routes_shapefile([samtrans_itp_id], ## samtrans
                                shared_utils.geography_utils.CA_NAD83Albers)

In [143]:
# shared_utils.utils.geoparquet_gcs_export(st_routelines, GCS_FILE_PATH, 'sam_routelines')

In [141]:
sam_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}sam_routelines.parquet')

In [142]:
sam_routelines

,calitp_itp_id,calitp_url_number,shape_id,geometry
0,290,0,1100224,"LINESTRING (-217339.785 -31678.594, -217339.78..."
1,290,1,1100236,"LINESTRING (-217339.785 -31678.594, -217339.78..."
2,290,0,1100237,"LINESTRING (-220663.991 -43864.472, -220663.99..."
3,290,1,1100256,"LINESTRING (-218302.184 -44113.771, -218302.18..."
4,290,1,1120134,"LINESTRING (-217098.080 -34074.660, -217098.08..."
...,...,...,...,...
438,290,1,SamTrans79:ECR1064,"LINESTRING (-216685.505 -31709.510, -216679.32..."
439,290,1,SamTrans79:ECR1065,"LINESTRING (-216685.505 -31709.510, -216679.32..."
440,290,1,SamTrans79:ECR1072,"LINESTRING (-191349.232 -61446.984, -191332.05..."
441,290,1,SamTrans79:FLXP0056,"LINESTRING (-220663.991 -43864.472, -220670.53..."


In [167]:
# trips_positions_joined >> filter(_.route_id == 'ECR')

In [170]:
example_trip = (trips_positions_joined
 >> filter(_.trip_key == 9182511101216666422)
)

In [191]:
importlib.reload(rt)

<module 'rt_analysis2' from '/home/jovyan/data-analyses/rt_delay/rt_analysis2.py'>

In [192]:
# example_trip[['calitp_url_number_x', 'calitp_url_number_y']] ## RT and static numbers, these need not match?

In [193]:
## gums up for 294...

In [194]:
example_vp_trip = rt.VehiclePositionsInterpolator(example_trip, sam_routelines)

sc_called
(422, 10)
check location data for trip 9182511101216666422
sc_called
(111, 16)


In [198]:
# example_vp_trip.detailed_speed_map()